In [1]:
# Load libraries
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
#from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.decomposition import RandomizedPCA
#from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from collections import Counter
#from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
# global import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import time
#from imblearn.over_sampling import RandomOverSampler


In [2]:
data = spark.table('npi_tax_data_csv') # spark.sql("SELECT * FROM " + npi_tax_data_csv)
dataDF = data.toPandas()
data10 = dataDF.loc[dataDF['from_tinytax'].isin(['from_tinytax', 'Optometrist', 'Anesthesiology', 'Psychiatry', 'Cardiovascular', 'Gastroenterology', 'Neurology', 'Dermatology','Pulmonary','Nephrology','Podiatrist'])]
from sklearn.model_selection import train_test_split
train, test = train_test_split(data10, test_size=0.2)

# 5 Build Models
We don’t know which algorithms would be good on this problem or what configurations to use. We get an idea from the plots that some of the classes are partially linearly separable in some dimensions, so we are expecting generally good results.

Let’s evaluate 5 different algorithms:

Logistic Regression.
Decision Tree & Random Forest (RF).
Naive Bayes.
Support Vector Machines (SVM).
k-Nearest Neighbors (kNN).
Linear Discriminant Analysis (LDA)
Quadratic Discriminant Analysis (QDA)
Boosting.
Classification and Regression Trees (CART).
https://docs.microsoft.com/en-us/azure/machine-learning/studio/algorithm-choice https://www.cs.princeton.edu/~schapire/talks/picasso-minicourse.pdf

This is a good mixture of simple linear (LDA), nonlinear (CART, kNN, Logistic) and complex nonlinear methods (SVM, RF). We reset the random number seed before reach run to ensure that the evaluation of each algorithm is performed using exactly the same data splits. It ensures the results are directly comparable.

## One-vs-All SGD for Multi-classification
The basic idea is to change multiple classes into two classes, and construct one logistic classifier for each class. We set the value of y (label，from_npi_tax_cd) of one class to 1, and 0 for other classes.

### Firstly test the gradient and loss function
Set the label of the first class to be one, and 0 for others

https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff https://www.geeksforgeeks.org/multiclass-classification-using-scikit-learn/

https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff

### 1) Optometrist vs non-Optometrist

Let's simplify the problem for now and only try to identify one taxonomy: Optometrist. So the problem is simplified to distinguishing between Optometrist and non-Optometrist.

In [5]:
X_train = train.iloc[:,2:]
print (X_train.head(3))
y_train = train.iloc[:,1]
print (y_train.head(3))
X_test = test.iloc[:,2:]
y_test = test.iloc[:,1]

Addiction Adolescent ... Urology Vascular_Interventional
1138494 0 0 ... 25 0
1151272 0 0 ... 417 0
286714 0 0 ... 0 0

[3 rows x 82 columns]
1138494 Optometrist
1151272 Gastroenterology
286714 Anesthesiology
Name: from_tinytax, dtype: object

In [6]:
y_train_Optometrist = (y_train == 'Optometrist')
y_test_Optometrist = (y_test == 'Optometrist')

In [7]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state = 42)
sgd_clf.fit(X_train, y_train_Optometrist)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
Out[34]: SGDClassifier(alpha=0.0001, average=False, class_weight=None,
 early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
 l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
 n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
 power_t=0.5, random_state=42, shuffle=True, tol=None,
 validation_fraction=0.1, verbose=0, warm_start=False)

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_Optometrist, cv = 3, scoring = "accuracy")

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
Out[35]: array([0.94698002, 0.95784056, 0.9617386 ])

Confusion Matrix:

In [10]:
from sklearn.model_selection import cross_val_predict
y_test_pred = cross_val_predict(sgd_clf, X_test, y_test_Optometrist, cv = 3)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_Optometrist, y_test_pred)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
Out[44]: array([[27659, 3727],
 [ 711, 7539]])

Precision and Recall

In [12]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test_Optometrist, y_test_pred)

Out[45]: 0.669181608379194

In [13]:
recall_score(y_test_Optometrist, y_test_pred)

Out[46]: 0.9138181818181819

In [14]:
from sklearn.metrics import f1_score
f1_score(y_test_Optometrist, y_test_pred)

Out[47]: 0.772596843615495

## Multi-class Classification
### OvO with SGDClassifier:

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
Out[52]: array([0.78736448, 0.76098927, 0.77921808])

In [17]:
sgd_clf.fit(X_train, y_train)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
Out[54]: SGDClassifier(alpha=0.0001, average=False, class_weight=None,
 early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
 l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
 n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
 power_t=0.5, random_state=42, shuffle=True, tol=None,
 validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
conf_mx = confusion_matrix(y_train, y_train_pred)
print (conf_mx)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
[[22871 1088 247 3272 261 488 244 152 1250 349]
 [ 318 11316 165 999 1021 460 109 156 2518 332]
 [ 437 140 8365 225 92 122 202 174 420 24]
 [ 2356 225 231 7724 238 177 34 31 937 88]
 [ 106 454 25 155 7249 53 30 33 418 86]
 [ 605 959 252 450 176 7141 114 182 1762 239]
 [ 1090 570 901 271 175 301 27709 513 1422 85]
 [ 718 657 344 233 188 202 285 3936 1349 33]
 [ 1219 813 242 249 239 655 345 378 13790 143]
 [ 156 928 51 469 383 360 43 55 1387 5308]]

In [19]:
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()
display()

In [20]:
y_test_pred = cross_val_predict(sgd_clf, X_test, y_test, cv=3)
conf_mx_test = confusion_matrix(y_test, y_test_pred)
print (conf_mx_test)

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
[[6241 223 81 284 55 243 61 56 36 349]
 [ 85 2524 63 12 82 567 15 54 33 891]
 [ 105 30 2068 10 4 77 60 47 9 78]
 [ 858 111 88 1287 41 254 26 10 31 289]
 [ 28 123 9 2 1770 53 10 10 7 157]
 [ 190 184 75 10 21 2057 12 42 62 212]
 [ 236 159 230 15 25 237 6838 247 69 194]
 [ 175 165 116 4 28 190 48 1054 60 145]
 [ 323 403 137 17 42 261 51 92 3049 212]
 [ 28 223 29 7 44 124 7 11 17 1852]]

In [21]:
plt.matshow(conf_mx_test, cmap=plt.cm.gray)
plt.show()
display()